# Agile Project Insights Dashboard - Analysis

This notebook demonstrates how to use the data processing and visualization components of the Agile Project Insights Dashboard.

In [ ]:
import pandas as pd
import os
import json
from pathlib import Path

# Import our modules
from src.data.processor import JiraDataProcessor
from src.visualization.charts import create_completion_donut, create_category_chart, generate_dashboard

# Check for sample data
sample_path = Path('./sample.csv')
if sample_path.exists():
    print(f"Reading sample data from {sample_path}")
else:
    print(f"Sample file not found at {sample_path}. Please provide a Jira CSV export.")
    
# Read the CSV file if it exists
if sample_path.exists():
    df = pd.read_csv(sample_path)
    
    # Print out all column names
    print("\nColumns in sample.csv:")
    print(df.columns.tolist())

Columns in sample.csv:
['Issue Type', 'Issue key', 'Issue id', 'Parent', 'Parent summary', 'Summary', 'Assignee', 'Assignee Id', 'Reporter', 'Reporter Id', 'Priority', 'Status', 'Resolution', 'Created', 'Updated', 'Due date', 'Original estimate', 'Description']


## Processing the Jira Data

Now we'll use the `JiraDataProcessor` class to process the data and extract insights.

In [ ]:
# Initialize the processor
if sample_path.exists():
    processor = JiraDataProcessor(file_path=str(sample_path))
    
    # Print available sprints
    print("\nAvailable Sprints:")
    for i, sprint in enumerate(processor.sprints):
        print(f"{i}: {sprint}")
    
    # Get current sprint data
    current_sprint_data = processor.get_sprint_data()
    
    # Print basic stats
    print("\nCurrent Sprint Stats:")
    print(f"Total issues: {len(current_sprint_data)}")
    print(f"Done issues: {len(current_sprint_data[current_sprint_data['Status'] == 'Done'])}")
    
    # Calculate metrics
    metrics = processor.calculate_sprint_metrics()
    
    # Print the metrics
    print("\nSprint Metrics:")
    for key, value in metrics.items():
        if key != 'blockers' and key != 'resource_utilization':
            print(f"{key}: {value}")
    
    # Print category breakdown
    print("\nCategory Breakdown:")
    print(f"Billable: {metrics['billable_hours']} hours")
    print(f"Product: {metrics['product_hours']} hours")
    print(f"Internal: {metrics['internal_hours']} hours")
    print(f"Other: {metrics['other_hours']} hours")
    
    # Print resource utilization
    print("\nResource Utilization:")
    for resource in metrics.get('resource_utilization', []):
        print(f"{resource['name']}: {resource['total_assigned']} hours assigned, {resource['completion_percentage']:.1f}% complete")
else:
    print("No sample data available.")

## Creating Visualizations

Now we'll generate some visualizations using the dashboard's plotting functions.

In [ ]:
# Only run this if we have data
if 'metrics' in locals():
    # Create completion donut chart
    completion_chart = create_completion_donut(
        metrics['completion_percentage'],
        metrics['total_story_points'],
        metrics['completed_story_points']
    )
    
    # Create category chart
    category_chart = create_category_chart(
        metrics['billable_hours'],
        metrics['product_hours'],
        metrics['internal_hours'],
        metrics['other_hours']
    )
    
    # Display the charts using Plotly
    import json
    import plotly.io as pio
    
    # Set the renderer to display in the notebook
    pio.renderers.default = "notebook"
    
    # Display completion chart
    print("Sprint Completion Chart:")
    pio.show(json.loads(completion_chart))
    
    # Display category chart
    print("Category Breakdown Chart:")
    pio.show(json.loads(category_chart))
else:
    print("No metrics available for visualization.")

## Full Dashboard Generation

Finally, we'll generate a complete dashboard with all charts.

In [ ]:
# Only run this if we have data
if 'processor' in locals():
    # Calculate velocity trend
    velocity_data = processor.calculate_velocity_trend()
    
    # Calculate scope change
    scope_change = processor.calculate_scope_change(current_sprint_data)
    
    # Project capacity
    projected_capacity = processor.project_future_capacity()
    
    # Generate dashboard
    dashboard = generate_dashboard(
        metrics,
        team_capacity=100,  # Example team capacity
        velocity_data=velocity_data,
        scope_change=scope_change,
        projected_capacity=projected_capacity
    )
    
    # Save the dashboard as a JSON file for reference
    with open('dashboard_example.json', 'w') as f:
        json.dump(dashboard, f, indent=2)
    
    print("Dashboard generated and saved to dashboard_example.json")
else:
    print("No processor available for dashboard generation.")